# Data Pre-Processing

### Stage 1
1. Load full dataset of Ethiopian schools and reduce to only the primary and secondary schools.
2. Keep only essential columns

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('../data/school_enrollment_18_19.csv')

In [3]:
print('Total Number of Schools: ', len(df))

Total Number of Schools:  41485


In [4]:
df1 = df.loc[(df['grade1_4'] > 0) | (df['grade5_8'] > 0) |
            (df['grade9_10'] > 0) | (df['grade11_12'] > 0)]

In [5]:
print('Total Number of Primary and Secondary Schools: ', len(df1))

Total Number of Primary and Secondary Schools:  37689


In [6]:
df1['grade_7_8'] =  df1['grade7_f'] + df1['grade7_m'] + df1['grade8_f'] + df1['grade8_m']

<ipython-input-6-45b8d2025b3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['grade_7_8'] =  df1['grade7_f'] + df1['grade7_m'] + df1['grade8_f'] + df1['grade8_m']


In [8]:
df1

,abelevel1_f,abelevel1_m,abelevel2_f,abelevel2_m,abelevel3_f,abelevel3_m,abelevel4_f,abelevel4_m,ctc_f,ctc_m,...,grade5_8_f,grade5_8_m,grade9_10,grade9_10_f,grade9_10_m,grade11_12,grade11_12_f,grade11_12_m,gr_offer,grade_7_8
43,0,0,0,0,0,0,0,0,0,0,...,0,0,1128,635,493,0,0,0,G. 9-10,0
109,0,0,0,0,0,0,0,0,0,0,...,1160,699,0,0,0,0,0,0,G.1-8,1026
110,0,0,0,0,0,0,0,0,0,0,...,774,492,0,0,0,0,0,0,G.1-8,649
111,0,0,0,0,0,0,0,0,0,0,...,395,320,0,0,0,0,0,0,G.1-8,430
112,0,0,0,0,0,0,0,0,0,0,...,123,119,0,0,0,0,0,0,G.1-8,142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,G.1-4,0
41481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,G.1-4,0
41482,0,0,0,0,0,0,0,0,0,0,...,0,0,279,174,105,0,0,0,G. 9-10,0
41483,0,0,0,0,0,0,0,0,0,0,...,0,0,117,55,62,0,0,0,G. 9-10,0


In [21]:
# Keep only the essential columns.
df2 = df1[['school_name','urban_rural','admin_code', 'grade1_4', 'grade5_8', 'grade_7_8', 'grade9_10', 
     'grade11_12','gr_offer']]

### Stage 2
 1. Load in base GIS locational file. 
 2. Cross check the administrative units with the correct shape files and find the correct region, zone and woreda (district)

In [10]:
df = pd.read_csv('../data/base_file.csv')
df.shape

(26227, 8)

In [11]:
x = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon,df.lat))

In [12]:
def check_country(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_region = gdf['ADM0_PCODE'].iloc[count]
            return new_region
        else:
            count +=1
            
def check_region(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_region = gdf['ADM1_PCODE'].iloc[count]
            return new_region
        else:
            count +=1
            
def check_zone(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_zone = gdf['ADM2_PCODE'].iloc[count]
            return new_zone
        else:
            count +=1
            
def check_woreda(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_woreda = gdf['ADM3_PCODE'].iloc[count]
            return new_woreda
        else:
            count +=1

In [13]:
# read in the shape files in geojson format. 
gdf_country = gpd.read_file('../eth_shape_files/json/eth_admin0v2.json')
gdf_region = gpd.read_file('../eth_shape_files/json//eth_admin1v2.json')
gdf_zone = gpd.read_file('../eth_shape_files/json//eth_admin2v2.json')
gdf_woreda = gpd.read_file('../eth_shape_files/json//eth_admin3v2.json')

In [14]:
# Create placeholder columns
x['ADM0_PCODE'] = ''
x['ADM1_PCODE'] = ''
x['ADM2_PCODE'] = ''
x['ADM3_PCODE'] = ''

In [15]:
# Run function on all GPS coordinates in dataset for country

i = 0
count = 0
while i < len(x):
        bb = check_country(x.iloc[i], gdf_country)
        x.loc[i, 'ADM0_PCODE'] = bb
        i +=1
        
# Run function on all GPS coordinates in dataset for region

i = 0
while i < len(x):
        bb = check_region(x.iloc[i], gdf_region)
        x.loc[i, 'ADM1_PCODE'] = bb
        i +=1

# Run function on all GPS coordinates in dataset for zone

i = 0
while i < len(x):
        bb = check_zone(x.iloc[i], gdf_zone)
        x.loc[i, 'ADM2_PCODE'] = bb
        i +=1
        
# Run function on all GPS coordinates in dataset for woreda

i = 0
while i < len(x):
        bb = check_woreda(x.iloc[i], gdf_woreda)
        x.loc[i, 'ADM3_PCODE'] = bb
        i +=1

In [16]:
# Review the dataset again and observe the differences between the administrative boundaries. 
# The original region, zone and woreda can be deleted at this stage and replaced with the corrected admin names.
# FYI Woreda = District in Ethiopia.

x.head()

,region,zone,woreda,school_name,admin_code,geometry,lat,lon,ADM0_PCODE,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE
0,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Tekort,S0306020712,POINT (37.98730 11.09990),11.099900,37.98730,ET,ET03,ET0306,ET030602
1,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Boda Kidanemihiret,S0306021792,POINT (37.69205 11.05947),11.059470,37.69205,ET,ET03,ET0306,ET030602
2,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Meklit,S0306020332,POINT (37.79732 11.07784),11.077843,37.79732,ET,ET03,ET0306,ET030602
3,Amhara,Misrak Gojjam,Sedie,Grarema,S0306020292,POINT (37.91040 10.74920),10.749200,37.91040,ET,ET03,ET0306,ET030602
4,Amhara,Misrak Gojjam,Sedie,Jema,S0306025422,POINT (37.80780 10.88000),10.880000,37.80780,ET,ET03,ET0306,ET030602


In [17]:
# Merge datasets basdd on the administrative codes. In this case, it is the PCODE

x= x.merge(gdf_woreda[['ADM0_PCODE', 'ADM0_EN', 'ADM1_PCODE', 'ADM1_EN', 'ADM2_PCODE', 'ADM2_EN', 'ADM3_PCODE', 'ADM3_EN']], 
             how='inner', on=['ADM0_PCODE','ADM1_PCODE', 'ADM2_PCODE', 'ADM3_PCODE'])

In [18]:
x.head()

,region,zone,woreda,school_name,admin_code,geometry,lat,lon,ADM0_PCODE,ADM1_PCODE,ADM2_PCODE,ADM3_PCODE,ADM0_EN,ADM1_EN,ADM2_EN,ADM3_EN
0,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Tekort,S0306020712,POINT (37.98730 11.09990),11.099900,37.98730,ET,ET03,ET0306,ET030602,Ethiopia,Amhara,East Gojam,Hulet Ej Enese
1,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Boda Kidanemihiret,S0306021792,POINT (37.69205 11.05947),11.059470,37.69205,ET,ET03,ET0306,ET030602,Ethiopia,Amhara,East Gojam,Hulet Ej Enese
2,Amhara,Misrak Gojjam,Hulet_Ej_Ene,Meklit,S0306020332,POINT (37.79732 11.07784),11.077843,37.79732,ET,ET03,ET0306,ET030602,Ethiopia,Amhara,East Gojam,Hulet Ej Enese
3,Amhara,Misrak Gojjam,Sedie,Grarema,S0306020292,POINT (37.91040 10.74920),10.749200,37.91040,ET,ET03,ET0306,ET030602,Ethiopia,Amhara,East Gojam,Hulet Ej Enese
4,Amhara,Misrak Gojjam,Sedie,Jema,S0306025422,POINT (37.80780 10.88000),10.880000,37.80780,ET,ET03,ET0306,ET030602,Ethiopia,Amhara,East Gojam,Hulet Ej Enese


 ### Stage 3

1. Merge the two datasets. 

In [24]:
df3 = x.merge(df2[['urban_rural','admin_code', 'grade1_4', 'grade5_8', 'grade_7_8', 'grade9_10', 
     'grade11_12','gr_offer']], how='inner', on=['admin_code'])

In [28]:
# Write out the CSV 
df3.to_csv('../data/clean_dataset.csv', index=True, encoding = 'utf-8')

In [33]:
len(df.loc[(df['grade1_4'] > 0) | (df['grade5_8'] > 0)])

34342